In [24]:
# ------------------------------
# AI Customer Support Backend (Deterministic Replies)
# ------------------------------
from transformers import pipeline
from functools import lru_cache

# ------------------------------
# Load models once (CPU)
# ------------------------------
# Sentiment analysis
sentiment_model = pipeline("sentiment-analysis")

# Category classification
category_model = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli"
)

# ------------------------------
# Define categories
# ------------------------------
CATEGORIES = [
    "billing issue",
    "technical problem",
    "account access",
    "refund request",
    "product question",
    "feedback",
    "complaint",
    "general inquiry",
    "spam"
]

# ------------------------------
# Sentiment Analysis
# ------------------------------
@lru_cache(maxsize=500)
def analyze_sentiment(text: str):
    r = sentiment_model(text)[0]
    return {
        "sentiment": r["label"].lower(),
        "confidence": round(r["score"], 3)
    }

# ------------------------------
# Category Classification
# ------------------------------
@lru_cache(maxsize=500)
def categorize(text: str):
    r = category_model(text, candidate_labels=CATEGORIES)
    return {
        "category": r["labels"][0],
        "confidence": round(r["scores"][0], 3)
    }

# ------------------------------
# Deterministic Auto-reply Generator
# ------------------------------
def generate_reply(message: str, category: str, sentiment: str):

    opener = {
        "positive": "Great to hear from you — thanks for reaching out!",
        "neutral": "Thanks for contacting us.",
        "negative": "I'm really sorry for the inconvenience — I understand how frustrating this can be."
    }.get(sentiment, "Thanks for contacting us.")

    replies = {
        "billing issue": (
            f"{opener} I see this relates to billing. "
            "Please share your invoice number or the charge you’re concerned about, "
            "and I’ll review it right away."
        ),
        "technical problem": (
            f"{opener} It looks like you're facing a technical issue. "
            "Could you describe what happens step-by-step or share any error messages? "
            "We’ll troubleshoot it together."
        ),
        "account access": (
            f"{opener} I understand you're having trouble accessing your account. "
            "Please confirm your registered email (do not share passwords), "
            "and I’ll help you get back in safely."
        ),
        "refund request": (
            f"{opener} I see you're asking about a refund. "
            "Please provide your order ID and the reason for the request, "
            "and I’ll review it according to our refund policy."
        ),
        "product question": (
            f"{opener} Happy to help with your product question! "
            "Let me know what you’re trying to do, "
            "and I’ll give you the most accurate guidance."
        ),
        "complaint": (
            f"{opener} Thank you for telling us — your feedback matters. "
            "Please explain what went wrong and how we can make this right."
        ),
        "feedback": (
            f"{opener} We really appreciate your feedback. "
            "It helps us improve our service for everyone."
        ),
        "general inquiry": (
            f"{opener} I’m glad to assist! "
            "Let me know a few more details so I can give you the best answer."
        ),
        "spam": (
            f"{opener} This message cannot be processed."
        )
    }

    return replies.get(category, replies["general inquiry"])

# ------------------------------
# Main AI Support System Function
# ------------------------------
def ai_support_system(message: str):
    if not message.strip():
        return {
            "category": {"category": "general inquiry", "confidence": 1.0},
            "sentiment": {"sentiment": "neutral", "confidence": 1.0},
            "auto_reply": "Could you please provide more details so I can assist you properly?"
        }

    sentiment = analyze_sentiment(message)
    category = categorize(message)
    auto_reply = generate_reply(message, category["category"], sentiment["sentiment"])

    return {
        "category": category,
        "sentiment": sentiment,
        "auto_reply": auto_reply
    }


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Device set to use cpu


In [25]:
import gradio as gr

# ------------------------------
# Define UI
# ------------------------------

def handle_message(message, ui_mode):
    if not message.strip():
        return "General Inquiry", "Neutral", "Please provide more details so I can assist you properly."

    result = ai_support_system(message)
    category = result["category"]["category"]
    sentiment = result["sentiment"]["sentiment"]
    reply = result["auto_reply"]

    # Format reply based on UI mode
    if ui_mode == "Simple Result View":
        formatted_reply = reply
    else:
        formatted_reply = f"Category: {category}\nSentiment: {sentiment}\n\nReply:\n{reply}"

    return category, sentiment, formatted_reply

# ------------------------------
# Gradio Blocks
# ------------------------------
with gr.Blocks(title="AI Customer Support") as demo:
    gr.Markdown("## AI Customer Support Assistant")
    gr.Markdown("Type a customer message below and get instant analysis & professional reply.\n")

    with gr.Row():
        message_input = gr.Textbox(
            label="Customer Message",
            placeholder="Type or paste the customer message here...",
            lines=6
        )

    ui_mode = gr.Radio(
        ["Simple Result View", "Detailed View"],
        label="UI Mode",
        value="Simple Result View"
    )

    submit_btn = gr.Button("Generate Response")

    category_out = gr.Textbox(label="Category")
    sentiment_out = gr.Textbox(label="Sentiment")
    reply_out = gr.Textbox(label="AI Reply", lines=8)

    submit_btn.click(
        fn=handle_message,
        inputs=[message_input, ui_mode],
        outputs=[category_out, sentiment_out, reply_out]
    )

# ------------------------------
# Launch app
# ------------------------------
if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4f28360ace55b05bbd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
